In [1]:
import requests


train_url = "https://github.com/shhuangmust/AI/raw/111-2/Google_Stock_Price_Train.csv"
test_url = "https://github.com/shhuangmust/AI/raw/111-2/Google_Stock_Price_Test.csv"


response_train = requests.get(train_url)
with open("Google_Stock_Price_Train.csv", "wb") as file:
    file.write(response_train.content)


response_test = requests.get(test_url)
with open("Google_Stock_Price_Test.csv", "wb") as file:
    file.write(response_test.content)

print("Files downloaded successfully.")


Files downloaded successfully.


In [2]:

# Import the libraries
import numpy as np
import matplotlib.pyplot as plt  # for plotting
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Import the training set
try:
    dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')  # 讀取訓練集
except FileNotFoundError:
    print("CSV file not found. Please make sure the file exists in the specified path.")
    raise

# Check the data structure (optional)
print(dataset_train.head())

# Select the "Open" column values
training_set = dataset_train.iloc[:, 1:2].values  # 取「Open」欄位值

# Feature Scaling
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Prepare the training data
X_train = []   # 預測點的前 60 天的資料
y_train = []   # 預測點
for i in range(60, len(training_set_scaled)):  # 使用 len(training_set_scaled) 以便自動調整
    X_train.append(training_set_scaled[i-60:i, 0])  # 取前60天資料
    y_train.append(training_set_scaled[i, 0])  # 預測當前點

# Convert to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the data to fit into RNN [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Check the shape of the data (optional)
print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')



       Date    Open    High     Low   Close      Volume
0  1/3/2012  325.25  332.83  324.97  663.59   7,380,500
1  1/4/2012  331.27  333.87  329.08  666.45   5,749,400
2  1/5/2012  329.83  330.75  326.89  657.21   6,590,300
3  1/6/2012  328.34  328.77  323.68  648.24   5,405,900
4  1/9/2012  322.04  322.29  309.46  620.76  11,688,800
X_train shape: (1198, 60, 1), y_train shape: (1198,)


In [ ]:
# Import the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
# Adding the output layer
regressor.add(Dense(units = 1))
# Compiling
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# 進行訓練
regressor.fit(X_train, y_train, epochs = 65, batch_size = 32)

Epoch 1/65
38/38 [==============================] - 5s 16ms/step - loss: 0.0353
Epoch 2/65
38/38 [==============================] - 1s 14ms/step - loss: 0.0061
Epoch 3/65
38/38 [==============================] - 1s 14ms/step - loss: 0.0057
Epoch 4/65
38/38 [==============================] - 1s 14ms/step - loss: 0.0054
Epoch 5/65
38/38 [==============================] - 1s 14ms/step - loss: 0.0056
Epoch 6/65
38/38 [==============================] - 1s 13ms/step - loss: 0.0046
Epoch 7/65
38/38 [==============================] - 1s 14ms/step - loss: 0.0050
Epoch 8/65
38/38 [==============================] - 1s 14ms/step - loss: 0.0050
Epoch 9/65
38/38 [==============================] - 1s 14ms/step - loss: 0.0048
Epoch 10/65
38/38 [==============================] - 1s 14ms/step - loss: 0.0043
Epoch 11/65
38/38 [==============================] - 1s 14ms/step - loss: 0.0043
Epoch 12/65
38/38 [==============================] - 1s 14ms/step - loss: 0.0043
Epoch 13/65
38/38 [==================

In [ ]:
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs) # Feature Scaling

X_test = []
for i in range(60, 80):  # timesteps一樣60； 80 = 先前的60天資料+2017年的20天資料
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))  # Reshape 成 3-dimension
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)  # to get the original scale


1/1 [==============================] - 1s 1s/step


In [3]:
# Importing required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Step 1: Load and preprocess your data (this is a summary of previous steps)
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Prepare the training data
X_train = []
y_train = []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Step 2: Build and train your model here (assumed you already have a model)

# Step 3: Make predictions (assuming you have a trained model)
# predicted_stock_price = model.predict(X_test)  # Make sure to create X_test based on your requirements

# Inverse scaling the predicted values (assuming you've defined real_stock_price)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)  # 反轉縮放

# Assume real_stock_price is defined (e.g., the test set values)
real_stock_price = pd.read_csv('Google_Stock_Price_Test.csv').iloc[:, 1:2].values
real_stock_price = sc.inverse_transform(real_stock_price)  # 反轉縮放

# Step 4: Visualising the results
plt.figure(figsize=(14, 7))  # 可以調整圖形大小
plt.plot(real_stock_price, color='red', label='Real Google Stock Price')  # 紅線表示真實股價
plt.plot(predicted_stock_price, color='blue', label='Predicted Google Stock Price')  # 藍線表示預測股價
plt.title('Google Stock Price Prediction', fontsize=16)  # 標題
plt.xlabel('Time', fontsize=14)  # X 軸標題
plt.ylabel('Google Stock Price', fontsize=14)  # Y 軸標題
plt.legend()
plt.grid()  # 添加網格以提高可讀性
plt.show()


NameError: name 'predicted_stock_price' is not defined